


###    Họ và tên: Hồ Công Lượng
###    MSSV: 19120572





# 1. Cài đặt môi trường

In [1]:
!pip install -U -q segmentation-models
!pip install -q tensorflow==2.1
!pip install -q keras==2.3.1
!pip install -q tensorflow-estimator==2.1.
## Imports libs
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Tải dữ liệu

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/drive/MyDrive/CNN_LSTM/IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# 3. Xử lý dữ liệu

In [5]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
exclude = string.punctuation
def remove_html_tags(text):
    re_html = re.compile('<.*?>')
    return re_html.sub(r'', text)

def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

df['review'] = df['review'].apply(remove_html_tags)
df['review'] = df['review'].apply(remove_punc)

In [6]:
# Tách 2 tập positive và negative riêng biệt
df_pos = np.array(df[df['sentiment'] == 'positive'].reset_index()['review'])
df_neg = np.array(df[df['sentiment'] == 'negative'].reset_index()['review'])

In [7]:
word2id = {}
id2word ={}
index = 1
# Word embedding
def word_embedding(reviews):
  global word2id
  global id2word
  global index
  temp = []
  for rev in reviews:
    rev = rev.replace('\n', "")
    ids = np.array([], dtype='int32')
    for word in rev.split(' '):
      word = word.lower()
      if word in word2id:
        ids = np.append(ids, word2id[word])
      else:
        if word != '':
          word2id[word] = index
          id2word[index] = word
          ids = np.append(ids, index)
          index = index + 1
    if len(ids) > 0:
      temp.append(ids)

  return temp

In [8]:
positive = word_embedding(df_pos)
negative = word_embedding(df_neg)

# 4. Tạo nhãn và chia data thành các tập train, test, val

In [9]:
y_pos = []
for i in positive:
  y_pos.append([1,0])
y_pos = np.array(y_pos)

y_neg = []
for i in negative:
  y_neg.append([0, 1])
y_neg = np.array(y_neg)

# Lấy 21000 samples cho tập train, 2000 samples cho tập test và 2000 samples cho tập validate4
# Train
pos_train = positive[0:21000]
neg_train = negative[0:21000]
y_pos_train = y_pos[0:21000]
y_neg_train = y_neg[0:21000]

# Test
pos_test = positive[21000:23000]
neg_test = negative[21000:23000]
y_pos_test = y_pos[21000:23000]
y_neg_test = y_neg[21000:23000]

# Validate
pos_val = positive[23000:25000]
neg_val = negative[23000:25000]
y_pos_val = y_pos[23000:25000]
y_neg_val = y_neg[23000:25000]

X_train = np.concatenate([pos_train, neg_train])
y_train = np.concatenate([y_pos_train, y_neg_train])

X_val = np.concatenate([pos_val, neg_val])
y_val = np.concatenate([y_pos_val, y_neg_val])

X_test = np.concatenate([pos_test, neg_test])
y_test = np.concatenate([y_pos_test, y_neg_test])

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_val.shape, y_val.shape)


(42000,) (42000, 2)
(4000,) (4000, 2)
(4000,) (4000, 2)


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


# 5. Train model

In [10]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D, LSTM
from keras.datasets import imdb
from keras import backend as K
from keras.optimizers import Adadelta
from keras.preprocessing import sequence as sq
from keras.layers import Dense, Dropout, Activation, Lambda,concatenate,Input,TimeDistributed,Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import keras.backend.tensorflow_backend as K

Using TensorFlow backend.


In [11]:
# set parameters:
max_features = len(word2id) + 1
maxlen = 400
embedding_dims = 200
nb_filter = 150
hidden_dims = 100

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

X_train = sq.pad_sequences(X_train, maxlen=maxlen)
X_test = sq.pad_sequences(X_test, maxlen=maxlen)
X_val = sq.pad_sequences(X_val, maxlen=maxlen)

# Build model
model = Sequential()

input_layer = Input(shape=(maxlen,),dtype='int32', name='main_input')
emb_layer = Embedding(max_features,
                      embedding_dims,
                      input_length=maxlen
                      )(input_layer)
def max_1d(X):
    return K.max(X, axis=1)

# 2 filter có filter_length=2
con2_1_layer = Convolution1D(nb_filter=nb_filter,
                    filter_length=2,
                    border_mode='valid',
                    activation='relu',
                    subsample_length=1)(emb_layer)

pool_con2_1_layer = Lambda(max_1d, output_shape=(nb_filter,))(con2_1_layer)

con2_2_layer = Convolution1D(nb_filter=nb_filter,
                    filter_length=2,
                    border_mode='valid',
                    activation='relu',
                    subsample_length=1)(emb_layer)

pool_con2_2_layer = Lambda(max_1d, output_shape=(nb_filter,))(con2_2_layer)

# 2 filter có filter_length=3
con3_1_layer = Convolution1D(nb_filter=nb_filter,
                    filter_length=3,
                    border_mode='valid',
                    activation='relu',
                    subsample_length=1)(emb_layer)

pool_con3_1_layer = Lambda(max_1d, output_shape=(nb_filter,))(con3_1_layer)

con3_2_layer = Convolution1D(nb_filter=nb_filter,
                    filter_length=3,
                    border_mode='valid',
                    activation='relu',
                    subsample_length=1)(emb_layer)

pool_con3_2_layer = Lambda(max_1d, output_shape=(nb_filter,))(con3_2_layer)

# Concatenate into CNN features
cnn_layer = concatenate([pool_con2_1_layer, pool_con2_2_layer, pool_con3_1_layer, pool_con3_2_layer])

#LSTM
x = Embedding(max_features, embedding_dims, input_length=maxlen)(input_layer)
lstm_layer = LSTM(128)(x)

# Concatenate CNN layer and LSTM layer
cnn_lstm_layer = concatenate([lstm_layer, cnn_layer])

dense_layer = Dense(hidden_dims*2, activation='sigmoid')(cnn_lstm_layer)
output_layer= Dropout(0.2)(dense_layer)
output_layer = Dense(2, trainable=True,activation='softmax')(output_layer)

42000 train sequences
4000 test sequences


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=150, kernel_size=2, strides=1, padding="valid")`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=150, kernel_size=2, strides=1, padding="valid")`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=150, kernel_size=3, strides=1, padding="valid")`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=150, kernel_size=3, strides=1, padding="valid")`


In [12]:
# Train model 
batch_size = 1024
nb_epoch = 5

model = Sequential()
model = Model(input=[input_layer], output=[output_layer])
adadelta = Adadelta(lr=1.0, rho=0.95, epsilon=1e-06)

model.compile(loss='categorical_crossentropy',
              optimizer="adamax",
              metrics=['accuracy'])

checkpoint = ModelCheckpoint('/content/drive/MyDrive/CNN_LSTM/weights.hdf5',
                              monitor='val_acc', verbose=0, save_weights_only=True,
                              mode='max')
model.fit(X_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          callbacks=[checkpoint],
          validation_data=(X_val, y_val))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 42000 samples, validate on 4000 samples
Epoch 1/5
42000/42000 [==============================] - 1119s 27ms/step - loss: 0.5787 - accuracy: 0.6760 - val_loss: 0.3451 - val_accuracy: 0.8558
Epoch 2/5
42000/42000 [==============================] - 1110s 26ms/step - loss: 0.2572 - accuracy: 0.8980 - val_loss: 0.2647 - val_accuracy: 0.8865
Epoch 3/5
42000/42000 [==============================] - 1115s 27ms/step - loss: 0.1626 - accuracy: 0.9413 - val_loss: 0.2645 - val_accuracy: 0.8955
Epoch 4/5
42000/42000 [==============================] - 1131s 27ms/step - loss: 0.1069 - accuracy: 0.9653 - val_loss: 0.3037 - val_accuracy: 0.8953
Epoch 5/5
42000/42000 [==============================] - 1127s 27ms/step - loss: 0.0698 - accuracy: 0.9797 - val_loss: 0.3855 - val_accuracy: 0.8850


In [13]:
# Save checkpoint 
model.save_weights("/content/drive/MyDrive/CNN_LSTM/weights.hdf5")

# 6. Đánh giá model trên tập test:

In [14]:
new_model = Model(input=[input_layer], output=[output_layer])

new_model.load_weights('/content/drive/MyDrive/CNN_LSTM/weights.hdf5')
model.compile(loss='categorical_crossentropy',
              optimizer="adamax",
              metrics=['accuracy'])
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)

print('acc = ', acc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


4000/4000 [==============================] - 27s 7ms/step
acc =  0.8872500061988831
